In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

In [3]:
# This file contains all the main external libs we'll use
import fastai
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.metrics import *
import pandas as pd
import numpy as np
import shutil
from IPython.display import FileLink, FileLinks

from sklearn.model_selection import train_test_split
import helper
import pyt

In [4]:
#PATH = "/home/nidhin/data/iceberg/"
PATH = os.path.expanduser("~/data/iceberg/")

In [5]:
!ls -l {PATH}

total 1975664
drwxrwxr-x  6 nidhin nidhin       4096 Jan 14 01:19 composites
drwxrwxr-x  2 nidhin nidhin       4096 Jan 21 15:51 models
-rw-rw-r--  1 nidhin nidhin     117951 Oct 23 17:27 sample_submission.csv
-rw-rw-r--  1 nidhin nidhin      38566 Jan  7 20:28 sample_submission.csv.7z
drwxrwxr-x  4 nidhin nidhin       4096 Jan 21 04:12 sub
drwxrwxr-x  2 nidhin nidhin       4096 Jan 21 02:25 test
-rw-rw-r--  1 nidhin nidhin      75819 Jan 21 02:25 test.csv
drwxrwxr-x  2 nidhin nidhin     286720 Jan 21 02:26 test-jpeg
-rw-rw-r--  1 nidhin nidhin 1521771850 Oct 23 17:27 test.json
-rw-rw-r--  1 nidhin nidhin  257127394 Jan  7 20:28 test.json.7z
drwxrwxr-x 16 nidhin nidhin       4096 Jan 21 02:35 tmp
drwxrwxr-x  4 nidhin nidhin       4096 Jan 21 02:25 train
-rw-rw-r--  1 nidhin nidhin      21714 Jan 21 02:25 train.csv
drwxrwxr-x  4 nidhin nidhin      61440 Jan 21 02:25 train-jpeg
-rw-rw-r--  1 nidhin nidhin  196313674 Oct 23 17:23 train.json
-rw-rw-r--  1 nidhin nidhin   44932785 Jan  7 20

In [6]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [7]:
train_df = pd.read_json(PATH+"/train.json")[["id","inc_angle","is_iceberg"]]

In [8]:
train_df.head()

,id,inc_angle,is_iceberg
0,dfd5f913,43.9239,0
1,e25388fd,38.1562,0
2,58b2aaa0,45.2859,1
3,4cfc3a18,43.8306,0
4,271f93f4,35.6256,0


In [9]:
#total number of images
num_images = len(train_df) 
sz=64   #image size
bs=32    #batch size
lr=0.001
lrs = [lr/9, lr/3, lr]
f_model = resnet34

In [12]:
def get_ensemble(nmodels):
    models = list()
    
    for i in range(nmodels):
        print(f'-----Training model: {i+1}--------')
        val_idx = get_cv_idxs(num_images, val_pct=0.1, seed=12345) #use 10% of train data as val data
        
        epochs = 5
        cycle_len = 1 
        cycle_mult = 2
        
        
        data = get_data_ens(64, i, val_idx)
        learn = ConvLearner.pretrained(f_model, data,precompute = False, ps=0.55)
        print('training for 64x64')
        learn.fit(lr, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        learn.unfreeze()
        learn.fit(lrs, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        
        print('training for 128x128')
        learn.set_data(get_data_ens(128, i, val_idx))
        learn.freeze()
        learn.fit(lr, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        learn.unfreeze()
        learn.fit(lrs, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        
        print('training for 150x150')
        learn.set_data(get_data_ens(150, i, val_idx))
        learn.freeze()
        learn.fit(lr, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        learn.unfreeze()
        learn.fit(lrs, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        
        print('training for 256x256')
        learn.set_data(get_data_ens(256, i, val_idx))
        learn.freeze()
        learn.fit(lr, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        learn.unfreeze()
        learn.fit(lrs, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        
        
        print('training for 128x128')
        learn.set_data(get_data_ens(128, i, val_idx))
        learn.freeze()
        learn.fit(lr, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        learn.unfreeze()
        learn.fit(lrs, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        
        
        print('training for 64x64')
        learn.set_data(get_data_ens(64, i, val_idx))
        learn.freeze()
        learn.fit(lr, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        learn.unfreeze()
        learn.fit(lrs, epochs, cycle_len=cycle_len, cycle_mult=cycle_mult)
        
        
        learn.save(f'ensem_model_{i}.weights')
        np.savez_compressed(f'{PATH}models/ensem_model_{i}_validx', val_idx=val_idx)
        models.append(learn)
        print(f'-----Training of model {i+1} complete----')
    return models
        
    
def get_data_ens(img_sz, model_index, val_idx):
    
    data = ImageClassifierData.from_csv(PATH,'train-jpeg', f'{PATH}train.csv',
                                     bs=bs, tfms=get_transform(model_index, img_sz),
                                     suffix='.jpeg',
                                     val_idxs=val_idx, test_name='test-jpeg')
    
    return  data

    
    
def get_transform(index, img_sz):
    f_model=resnet34
    index = (index%5)
    print(f'get_transform--{index}: {img_sz}')
    tfms = [
        tfms_from_model(f_model, img_sz, aug_tfms=transforms_basic, max_zoom=1.05),
        tfms_from_model(f_model, img_sz, aug_tfms=transforms_side_on, max_zoom=1.05),
        tfms_from_model(f_model, img_sz, aug_tfms=transforms_top_down, max_zoom=1.05),
        tfms_from_model(f_model, img_sz, aug_tfms=transforms_top_down, max_zoom=1.1),
        tfms_from_model(f_model, img_sz, aug_tfms=transforms_top_down, max_zoom=1.05, crop_type=CropType.RANDOM)]
    return tfms[index]

In [13]:
%time ens = get_ensemble(2)


-----Training model: 1--------
get_transform--0: 64
training for 64x64


HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

[ 0.       0.80446  0.62024  0.65   ]                      
[ 1.       0.69591  0.55519  0.725  ]                      
[ 2.       0.62873  0.5542   0.7    ]                      
[ 3.       0.57813  0.52206  0.75   ]                      
[ 4.       0.52864  0.50555  0.76875]                      
[ 5.       0.519    0.50788  0.75625]                      
[ 6.       0.4991   0.50587  0.7375 ]                      
[ 7.       0.51109  0.50756  0.74375]                      
[ 8.       0.5266   0.48767  0.76875]                      
[ 9.       0.49111  0.48641  0.775  ]                      
[ 10.        0.47775   0.48524   0.76875]                  
[ 11.        0.47592   0.48064   0.76875]                  
[ 12.        0.48273   0.48317   0.78125]                  
[ 13.        0.47794   0.48123   0.775  ]                  
[ 14.        0.4817    0.48109   0.78125]                  
[ 15.        0.46853   0.48154   0.76875]                  
[ 16.        0.4774    0.47748   0.76875

HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

[ 0.       0.71789  0.61816  0.64375]                      
[ 1.       0.62471  0.56994  0.70625]                      
[ 2.       0.56309  0.59803  0.65   ]                      
[ 3.       0.52356  0.51399  0.71875]                      
[ 4.       0.5055   0.5104   0.73125]                      
[ 5.       0.46003  0.49457  0.74375]                      
[ 6.       0.43084  0.49627  0.725  ]                      
[ 7.       0.43655  0.47357  0.7375 ]                      
[ 8.       0.42243  0.47493  0.73125]                      
[ 9.       0.40967  0.47378  0.75   ]                      
[ 10.        0.3616    0.49732   0.75   ]                  
[ 11.        0.36336   0.47701   0.74375]                  
[ 12.        0.34211   0.46204   0.7625 ]                  
[ 13.        0.35842   0.47783   0.75   ]                  
[ 14.        0.3402    0.43759   0.7625 ]                  
[ 15.        0.3229    0.49234   0.75625]                  
[ 16.        0.31616   0.45741   0.7625 

...

[ 0.       0.17007  0.27121  0.88125]                      
[ 1.       0.18962  0.3309   0.875  ]                      
[ 2.       0.16065  0.31072  0.88125]                      
[ 3.       0.16965  0.32454  0.88125]                      
[ 4.       0.19334  0.33396  0.86875]                      
[ 5.       0.16443  0.35444  0.86875]                      
[ 6.       0.15414  0.33223  0.86875]                      
[ 7.       0.13642  0.33393  0.875  ]                      
[ 8.       0.1268   0.38838  0.86875]                      
[ 9.       0.12464  0.37841  0.8625 ]                      
[ 10.        0.11373   0.35222   0.8625 ]                  
[ 11.        0.10676   0.41907   0.86875]                  
[ 12.        0.09659   0.41989   0.8625 ]                   
[ 13.        0.14433   0.35926   0.875  ]                   
[ 14.        0.1384    0.37984   0.88125]                  
[ 15.        0.19323   0.35625   0.86875]                  
[ 16.        0.14761   0.33631   0.862

HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

[ 0.       0.41795  0.51942  0.75   ]                      
[ 1.       0.41475  0.4743   0.76875]                      
[ 2.       0.37315  0.46211  0.775  ]                      
[ 3.       0.3752   0.45494  0.775  ]                      
[ 4.       0.36508  0.44717  0.78125]                      
[ 5.       0.35334  0.44548  0.7875 ]                      
[ 6.       0.35609  0.44065  0.7875 ]                      
[ 7.       0.36027  0.45421  0.75625]                      
[ 8.       0.34602  0.44659  0.775  ]                      
[ 9.       0.3435   0.43892  0.7875 ]                      
[ 10.        0.34081   0.43682   0.775  ]                  
[ 11.        0.34312   0.44131   0.775  ]                  
[ 12.        0.32835   0.43405   0.78125]                  
[ 13.        0.3217    0.43461   0.78125]                  
[ 14.        0.33172   0.43896   0.7875 ]                  
[ 15.        0.33734   0.44042   0.775  ]                  
[ 16.        0.3369    0.43582   0.775  

HBox(children=(IntProgress(value=0, description='Epoch', max=31), HTML(value='')))

[ 0.       0.31699  0.3936   0.825  ]                      
[ 1.       0.29619  0.39415  0.80625]                      
[ 2.       0.28503  0.39254  0.825  ]                      
[ 3.       0.27578  0.41126  0.81875]                      
[ 4.       0.2481   0.41863  0.825  ]                      
[ 5.       0.23774  0.40018  0.8125 ]                      
[ 6.       0.2653   0.42866  0.81875]                      
[ 7.       0.25795  0.42617  0.81875]                      
[ 8.       0.26647  0.44682  0.8125 ]                      
[ 9.       0.25729  0.41043  0.8375 ]                      
[ 10.        0.25347   0.46084   0.8    ]                  
[ 11.        0.23265   0.4733    0.8    ]                  
[ 12.        0.2091    0.47028   0.79375]                  
[ 13.        0.20474   0.46489   0.8125 ]                  
[ 14.        0.19741   0.45665   0.81875]                  
[ 15.        0.19869   0.46818   0.80625]                  
[ 16.        0.23317   0.46282   0.81875

# submission

In [14]:
ens

[Sequential(
   (0): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
   (2): ReLU(inplace)
   (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
   (4): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
       (relu): ReLU(inplace)
       (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
       (relu): ReLU(inplace)
       (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): Batc

In [16]:
#Find optimized Threshold
ens_val_probs=list()
#th_list=list()

df2 = pd.DataFrame({'image_name':ens[0].data.test_ds.fnames})
df2.head()

for (idx,mdl) in enumerate(ens):
    test_probs, y=mdl.TTA(is_test=True)
    print(test_probs.shape)
    test_probs = np.mean(np.exp(test_probs),axis=0)

    print(test_probs.shape)

    df2[f'm{idx}'] = test_probs[:,0]
    
    
    #th =opt_th(val_probs, y)
    #th_list.append(val_probs)
    #ens_val_probs.append(val_probs)
#print(th_list)   
#op_th = np.mean(th_list)
#print(op_th)

(5, 8424, 2)                                 
(8424, 2)
(5, 8424, 2)                                 
(8424, 2)


In [17]:
df2.head()

,image_name,m0,m1
0,test-jpeg/b2cd6017.jpeg,0.063703,0.337683
1,test-jpeg/f3fa0370.jpeg,0.000710,0.021244
2,test-jpeg/858fd565.jpeg,0.001487,0.004413
3,test-jpeg/9532a83b.jpeg,0.051714,0.125578
4,test-jpeg/6b189724.jpeg,0.006426,0.006195


In [20]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [21]:
df2['is_iceberg_max'] = df2.iloc[:, 1:3].max(axis=1)
df2['is_iceberg_min'] = df2.iloc[:, 1:3].min(axis=1)
df2['is_iceberg_mean'] = df2.iloc[:, 1:3].mean(axis=1)
df2['is_iceberg_median'] = df2.iloc[:, 1:3].median(axis=1)

In [22]:
df2['is_iceberg_base'] = df2['m0']
df2['is_iceberg'] = np.where(np.all(df2.iloc[:,1:3] > cutoff_lo, axis=1), 
                                    df2['is_iceberg_max'], 
                                    np.where(np.all(df2.iloc[:,1:4] < cutoff_hi, axis=1),
                                             df2['is_iceberg_min'], 
                                             df2['is_iceberg_base']))
df2['is_iceberg'] = np.clip(df2['is_iceberg'].values, 0.001, 0.999)


In [23]:
df2['diff'] = np.abs(df2['is_iceberg'] - df2['m0']) >0.1

In [25]:
df2[['image_name','is_iceberg']].head()

,image_name,is_iceberg
0,test-jpeg/b2cd6017.jpeg,0.063703
1,test-jpeg/f3fa0370.jpeg,0.001000
2,test-jpeg/858fd565.jpeg,0.001487
3,test-jpeg/9532a83b.jpeg,0.051714
4,test-jpeg/6b189724.jpeg,0.006195
